<a href="https://colab.research.google.com/github/vkjadon/openai/blob/main/04oai_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We can load the api key from `.env` file by importing `load_dotenv` method of `dotenv` package. We can install `dotenv` package as

> `pip install python-dotenv`

> `from dotenv import load_dotenv, find_dotenv`

`find_dotenv()` : This function searches up the directory tree from your current working directory until it locates the .env file.

`load_dotenv()` : This function reads the variables from the found .env file and loads them into your Python process's environment variables (specifically, os.environ).

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

client = OpenAI()

In case you are running the code on Google Colab, add your "OPENAI_API_KEY" key in the `Secrets` of the colab environment and call instantiate the `OpenAI` object as below

In [ ]:
!pip install -q faiss-cpu pypdf tiktoken

In [ ]:
from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

In [ ]:
from pypdf import PdfReader

reader = PdfReader(pdf_path)

full_text = ""
scanned_pages = []

for i, page in enumerate(reader.pages):
    page_text = page.extract_text()

    if page_text:
        full_text += page_text + "\n"
    else:
        scanned_pages.append(i)

print("Preview:\n", full_text[:1000])
print("Scanned pages:", scanned_pages)

In [ ]:
def chunk_text(text, chunk_size=800, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks

In [ ]:
chunks = chunk_text(full_text)
print("Total chunks:", len(chunks))

In [ ]:
import numpy as np

In [ ]:
def get_embeddings(texts):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=texts
    )
    return np.array([e.embedding for e in response.data])

In [ ]:
chunk_embeddings = get_embeddings(chunks)
print(chunk_embeddings.shape)

FAISS (Facebook AI Similarity Search) is an open-source library for efficient similarity search and clustering of dense, high-dimensional vectors. It is used to quickly find items in massive datasets that are "similar" to a given query item, solving performance bottlenecks that traditional databases face when dealing with unstructured data like images or text embeddings.

In [ ]:
import faiss

dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(chunk_embeddings)

print("FAISS index size:", index.ntotal)

In [ ]:
def retrieve_chunks(query, k=3):
    query_embedding = get_embeddings([query])
    distances, indices = index.search(query_embedding, k)
    return [chunks[i] for i in indices[0]]

In [ ]:
def build_context(query):
    retrieved = retrieve_chunks(query)
    return "\n\n".join(retrieved)

In [ ]:
def ask_rag(question):
  context = build_context(question)

  response = client.responses.create(
      model="gpt-4.1-mini",
      input=[
        {
          "role": "system",
          "content": "You are a helpful teacher."
        },
        {
          "role": "developer",
          "content": (
                        "Answer ONLY using the provided context. "
                        "If the answer is not present, say 'I do not know.'"
                    )
        },
        {
          "role": "user",
          "content": f"""
                      {context}

                      {question}
                      """
        }
        ]
      )
  return response.output_text


In [ ]:
question = "What is the material for a gear."
answer = ask_rag(question)

print(answer)
